# **05. Python Pandas II**

## **Homework Assignments / Coaching Session Doubts and Practice**

In [2]:
import numpy as np
import pandas as pd

## **Exercise 1** 

We have a dataframe that considers different salaries for Data positions in different countries:

    - Import the dataframe
    - Do an initial exploration - What do you see?
    - What's the compared mean Average Salary in ES, FR and the US? Find it out, using Pandas base exploratory funcitons.

In [3]:
df = pd.read_csv('input_data/salaries.csv')

In [4]:
df.sample(5)

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
8256,2022,SE,FT,Data Engineer,136994,USD,136994,US,100,US,M
7540,2022,SE,FT,Data Engineer,150000,USD,150000,US,0,US,M
3760,2023,SE,FT,Applied Scientist,260000,USD,260000,US,0,US,L
2952,2023,SE,FT,Decision Scientist,142200,USD,142200,US,0,US,M
1121,2023,MI,FT,Research Scientist,100000,USD,100000,US,0,US,M


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8805 entries, 0 to 8804
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           8805 non-null   int64 
 1   experience_level    8805 non-null   object
 2   employment_type     8805 non-null   object
 3   job_title           8805 non-null   object
 4   salary              8805 non-null   int64 
 5   salary_currency     8805 non-null   object
 6   salary_in_usd       8805 non-null   int64 
 7   employee_residence  8805 non-null   object
 8   remote_ratio        8805 non-null   int64 
 9   company_location    8805 non-null   object
 10  company_size        8805 non-null   object
dtypes: int64(4), object(7)
memory usage: 756.8+ KB


In [38]:
# we have some indication of remote, but still we see they are more or less equitative, employee residence and jobs
df.company_location.value_counts()

US    7576
GB     424
CA     205
ES     108
DE      72
      ... 
QA       1
HK       1
ZA       1
AD       1
MT       1
Name: company_location, Length: 74, dtype: int64

In [34]:
# Super unbalanced dataset as for data, but surprisingly lots of jobs for ES!
df.employee_residence.value_counts()

US    7527
GB     417
CA     204
ES     112
IN      66
      ... 
KW       1
EC       1
PE       1
IL       1
MT       1
Name: employee_residence, Length: 86, dtype: int64

**Getting mean Salaries in ES, FR, GB**

In [5]:
df_result = pd.DataFrame()

In [44]:
df_result["ES"] = df[df.company_location.isin(['ES'])].describe().loc['mean']
df_result["FR"] = df[df.company_location.isin(['FR'])].describe().loc['mean']
df_result["GB"] = df[df.company_location.isin(['GB'])].describe().loc['mean']
df_result["US"] = df[df.company_location.isin(['US'])].describe().loc['mean']
df_result["JP"] = df[df.company_location.isin(['JP'])].describe().loc['mean']

In [47]:
# surprising or not that much?
df_result

,ES,FR,GB,US,JP
work_year,2022.472222,2022.020408,2022.688679,2022.783923,2021.125
salary,53252.527778,74841.224490,92992.955189,160612.771515,3264000.000
salary_in_usd,56418.703704,78672.163265,105983.158019,158329.921067,110821.625
remote_ratio,36.111111,64.285714,37.735849,36.932418,50.000


# **Exercise 2**

Obtain a DataFrame that, per country, shows the mean salary, in local currency, and the maximum and minimum salaries, all per experience level.

In [48]:
df.columns

Index(['work_year', 'experience_level', 'employment_type', 'job_title',
       'salary', 'salary_currency', 'salary_in_usd', 'employee_residence',
       'remote_ratio', 'company_location', 'company_size'],
      dtype='object')

In [52]:
grouped = pd.DataFrame()
grouped['mean'] = df.groupby(by = ['company_location', 'experience_level']).salary.mean()
grouped['max'] = df.groupby(by = ['company_location', 'experience_level']).salary.max()
grouped['min'] = df.groupby(by = ['company_location', 'experience_level']).salary.min()

In [12]:
grouped_data = df.groupby(['employee_residence', 'experience_level'])

# Calculate mean, max, and min salaries for each group
result_df = grouped_data.agg({'salary': ['mean', 'max', 'min']}).reset_index()

In [53]:
grouped

mean       max     min
company_location experience_level                                 
AD               MI                 47000.000000     47000   47000
AE               MI                115000.000000    115000  115000
                 SE                 92500.000000    120000   65000
AM               MI                 50000.000000     50000   50000
AR               EN                 50000.000000     50000   50000
...                                          ...       ...     ...
US               EN                103500.841424    281700   20000
                 EX                196326.487069    416000   55000
                 MI                139839.591276  11000000   20000
                 SE                166664.039676   4000000   25000
ZA               MI                627000.000000    627000  627000

[156 rows x 3 columns]

In [54]:
grouped.loc[['ES', 'FR', 'GB']]

mean     max    min
company_location experience_level                              
ES               EN                 28800.000000   38000  20000
                 EX                 64500.000000   70000  59000
                 MI                 54041.250000   95000  30000
                 SE                 54302.376623  253750  35000
FR               EN                 42800.000000  110000  20000
                 MI                 61717.391304  110000  37000
                 SE                113732.500000  225000  45000
GB               EN                 55351.851852  160800  27000
                 EX                153768.777778  228856  80000
                 MI                 72742.343478  350000  30000
                 SE                123731.114094  325000  35000

# **Exercise 3**

There are many types of Data Jobs available.

Create the same grouping as before, plus an added dimension, "type of job", created from scratch considering:

    - If the title considers 'ML', 'Machine Learning', 'Science' or 'AI', then the category should be 'ML-Related'
    - Else, if the title considers 'BI' or 'Business Intelligence', then it should be 'BI-Related'
    - Else, if the title consders 'Analy', it should be 'Analyst'
    - Else, if the title considers 'ETL' or 'Data Engineer' it should be 'Data Engineer'
    - Else, if the title considers 'Cloud' or 'Architect', it should be 'Data/Cloud Architect'
    - Else, the category should be 'Others'.
    
Think what flow of transformations you need to achieve this. Plus, take into account the elses as mentioned before!

In [55]:
# First, create the grouping:

def grouping(x):
    if any([i in x for i in ['ML', 'Machine Learning', 'Science','AI']]):
        return 'ML-Related'
    elif any([i in x for i in ['BI', 'Business Intelligence']]):
        return 'BI-Related'
    elif any([i in x for i in ['Analy']]):
        return 'Analyst'
    elif any([i in x for i in ['ETL', 'Data Engineer']]):
        return 'Data Engineer'
    elif any([i in x for i in ['Cloud', 'Architect']]):
        return 'Data Architect'
    else:
        return 'Others'

In [56]:
df.columns

Index(['work_year', 'experience_level', 'employment_type', 'job_title',
       'salary', 'salary_currency', 'salary_in_usd', 'employee_residence',
       'remote_ratio', 'company_location', 'company_size'],
      dtype='object')

In [57]:
df['type_of_job'] = df.job_title.apply(grouping)

In [58]:
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,type_of_job
0,2023,EX,FT,Data Science Director,212000,USD,212000,US,0,US,M,ML-Related
1,2023,EX,FT,Data Science Director,190000,USD,190000,US,0,US,M,ML-Related
2,2023,MI,FT,Business Intelligence Engineer,35000,GBP,43064,GB,0,GB,M,BI-Related
3,2023,MI,FT,Business Intelligence Engineer,35000,GBP,43064,GB,0,GB,M,BI-Related
4,2023,SE,FT,Machine Learning Engineer,245700,USD,245700,US,0,US,M,ML-Related


**Now, let's get the grouping**

In [59]:
grouped = pd.DataFrame()
grouped['mean'] = df.groupby(by = ['company_location', 'experience_level', 'type_of_job']).salary.mean()
grouped['max'] = df.groupby(by = ['company_location', 'experience_level', 'type_of_job']).salary.max()
grouped['min'] = df.groupby(by = ['company_location', 'experience_level', 'type_of_job']).salary.min()

In [60]:
grouped.loc[['ES', 'FR', 'GB']]

mean     max  \
company_location experience_level type_of_job                             
ES               EN               Analyst          20000.000000   20000   
                                  Data Engineer    38000.000000   38000   
                                  ML-Related       28000.000000   30000   
                                  Others           30000.000000   30000   
                 EX               Data Engineer    70000.000000   70000   
                                  ML-Related       59000.000000   59000   
                 MI               Analyst          35000.000000   40000   
                                  Data Engineer    67615.384615   95000   
                                  ML-Related       35597.500000   40000   
                                  Others           41120.000000   55000   
                 SE               Analyst          43644.875000   52000   
                                  Data Engineer    56960.454545   85000   
                                  ML-Related       57500.000000   60000   
                                  Others           57112.500000  253750   
FR               EN               Analyst          38500.000000   50000   
                                  ML-Related       20000.000000   20000   
                                  Others           47285.714286  110000   
                 MI               Analyst          41000.000000   41000   
                                  BI-Related      100000.000000  100000   
                                  Data Engineer    69388.888889  110000   
                                  ML-Related       55000.000000   70000   
                                  Others           54400.000000   90000   
                 SE               ML-Related      118100.000000  225000   
                                  Others          106453.333333  225000   
GB               EN               Analyst          38166.666667   50000   
                                  Data Engineer    49200.000000  144000   
                                  ML-Related       33375.000000   45000   
                                  Others           91428.571429  160800   
                 EX               Data Engineer   129814.000000  205785   
                                  ML-Related      158631.666667  200000   
                                  Others          162099.000000  228856   
                 MI               Analyst          75768.206897  350000   
                                  BI-Related       46250.000000   75000   
                                  Data Architect   67500.000000   70000   
                                  Data Engineer    65485.915493  120000   
                                  ML-Related       85516.548387  300000   
                                  Others           73682.343750  280000   
                 SE               Analyst         119204.900000  325000   
                                  BI-Related       65375.000000   80000   
                                  Data Architect  100555.444444  138000   
                                  Data Engineer    97515.411765  199000   
                                  ML-Related      161974.592593  258800   
                                  Others          138929.777778  277300   

                                                     min  
company_location experience_level type_of_job             
ES               EN               Analyst          20000  
                                  Data Engineer    38000  
                                  ML-Related       26000  
                                  Others           30000  
                 EX               Data Engineer    70000  
                                  ML-Related       59000  
                 MI               Analyst          30000  
                                  Data Engineer    45000  
                                  ML-Related       30390  
                                  

# **Exercise 4**

Get a list of the unique countries from the previous dataset of salaries.

Now, import the cost of living dataset.

From the following countries: ('FR', 'US', 'UK', 'ES', 'JP', 'IT', 'DE', 'UA'), compare the average salary per position and type of job, as obtained before, with the cost of living, assuming that they live in the cities of FR: Paris; UK: London, USA: NYC, San Francisco, Austin; ES: Barcelona, Madrid; IT: Milan; JP: Tokyo, Osaka; DE: Berlin, Munich, Frankfurt, Köln; UA: Kiev. Do so by dividing the medium obtained salary by the cost of living index.

For an entry level position, what's the most expensive city/country? And for a Mid one?


In [61]:
df2 = pd.read_csv('input_data/costofliving.csv')

In [62]:
df2.head()

,Rank,City,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index
0,NaN,"Hamilton, Bermuda",149.02,96.10,124.22,157.89,155.22,79.43
1,NaN,"Zurich, Switzerland",131.24,69.26,102.19,136.14,132.52,129.79
2,NaN,"Basel, Switzerland",130.93,49.38,92.70,137.07,130.95,111.53
3,NaN,"Zug, Switzerland",128.13,72.12,101.87,132.61,130.93,143.40
4,NaN,"Lugano, Switzerland",123.99,44.99,86.96,129.17,119.80,111.96


In [68]:
# FIlter to obtain just relevant countries
grouped_filtered = grouped.loc[['ES', 'IT', 'FR', 'GB', 'US', 'JP', 'UA', 'DE']]

In [76]:
# Same for the cost of living dataset
df2['city_extracted'] = df2.City.apply(lambda x:x.split(', ')[0])
df2 = df2[df2.city_extracted.isin(['Paris', 'Madrid', 'Barcelona', 'London', 'New York', 'San Francisco', 'Austin', 'Milan', 'Tokyo', 'Osaka', 'Berlin', 'Munich', 'Frankfurt', 'Kiev', 'Cologne'])]

In [77]:
df2

,Rank,City,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index,city_extracted
13,NaN,"New York, NY, United States",100.00,100.00,100.00,100.00,100.00,100.00,New York
21,NaN,"San Francisco, CA, United States",93.91,108.42,100.72,97.05,93.40,133.16,San Francisco
33,NaN,"London, United Kingdom",85.62,76.51,81.35,64.08,89.41,88.79,London
34,NaN,"Tokyo, Japan",85.61,42.71,65.50,94.94,52.26,88.58,Tokyo
37,NaN,"Paris, France",84.35,48.79,67.68,83.65,81.64,80.20,Paris
71,NaN,"Munich, Germany",77.52,47.34,63.37,66.77,76.82,95.16,Munich
85,NaN,"Osaka, Japan",76.71,29.13,54.41,82.24,44.42,81.58,Osaka
116,NaN,"Milan, Italy",74.80,44.61,60.65,62.83,80.37,53.96,Milan
149,NaN,"Frankfurt, Germany",72.78,39.15,57.02,58.44,66.41,106.89,Frankfurt
185,NaN,"London, Canada",70.49,36.54,54.57,64.74,79.20,100.36,London


What we see is that stand out is that we selected a London in Canada, and that we do not have a country code to link it with our main df table. We are ging to have to take care of that.

In [83]:
def city_code(x):
    dmap = {'Spain':'ES', 'France':'FR', 'United States' : 'US', 'United Kingdom':'GB', 'Japan':'JP', 'Ukraine':'UA', 'Germany':'DE', 'Italy':'IT'}
    
    try:
        return dmap[x.split(', ')[-1]] # if it raises an error, there's a key not there (e.g. Canada), defaults to null
    except:
        pass

In [84]:
df2['country_code'] = df2.City.apply(city_code)

**Now, join**

In [87]:
df.columns

Index(['work_year', 'experience_level', 'employment_type', 'job_title',
       'salary', 'salary_currency', 'salary_in_usd', 'employee_residence',
       'remote_ratio', 'company_location', 'company_size', 'type_of_job'],
      dtype='object')

In [93]:
merged = grouped.reset_index().merge(df2, left_on = 'company_location', right_on = 'country_code')

In [96]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 208 entries, 0 to 207
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   company_location                208 non-null    object 
 1   experience_level                208 non-null    object 
 2   type_of_job                     208 non-null    object 
 3   mean                            208 non-null    float64
 4   max                             208 non-null    int64  
 5   min                             208 non-null    int64  
 6   Rank                            0 non-null      float64
 7   City                            208 non-null    object 
 8   Cost of Living Index            208 non-null    float64
 9   Rent Index                      208 non-null    float64
 10  Cost of Living Plus Rent Index  208 non-null    float64
 11  Groceries Index                 208 non-null    float64
 12  Restaurant Price Index          208 

In [97]:
merged['liveability'] = merged['mean']/merged['Cost of Living Index']

In [100]:
merged[merged.experience_level.isin(['MI'])].sort_values('liveability').head(26)

,company_location,experience_level,type_of_job,mean,max,min,Rank,City,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index,city_extracted,country_code,liveability
124,IT,MI,Others,24000.000000,24000,24000,NaN,"Milan, Italy",74.80,44.61,60.65,62.83,80.37,53.96,Milan,IT,320.855615
95,FR,MI,Analyst,41000.000000,41000,41000,NaN,"Paris, France",84.35,48.79,67.68,83.65,81.64,80.20,Paris,FR,486.069947
110,GB,MI,BI-Related,46250.000000,75000,35000,NaN,"London, United Kingdom",85.62,76.51,81.35,64.08,89.41,88.79,London,GB,540.177529
76,ES,MI,Analyst,35000.000000,40000,30000,NaN,"Barcelona, Spain",61.15,30.60,46.83,49.39,61.89,70.83,Barcelona,ES,572.363042
123,IT,MI,ML-Related,43200.000000,43200,43200,NaN,"Milan, Italy",74.80,44.61,60.65,62.83,80.37,53.96,Milan,IT,577.540107
80,ES,MI,ML-Related,35597.500000,40000,30390,NaN,"Barcelona, Spain",61.15,30.60,46.83,49.39,61.89,70.83,Barcelona,ES,582.134096
77,ES,MI,Analyst,35000.000000,40000,30000,NaN,"Madrid, Spain",59.01,31.95,46.32,49.14,62.31,74.83,Madrid,ES,593.119810
81,ES,MI,ML-Related,35597.500000,40000,30390,NaN,"Madrid, Spain",59.01,31.95,46.32,49.14,62.31,74.83,Madrid,ES,603.245213
24,DE,MI,Analyst,47000.000000,55000,39000,NaN,"Munich, Germany",77.52,47.34,63.37,66.77,76.82,95.16,Munich,DE,606.295150
28,DE,MI,BI-Related,48000.000000,48000,48000,NaN,"Munich, Germany",77.52,47.34,63.37,66.77,76.82,95.16,Munich,DE,619.195046
